# Tagging vessel arrival/departure points

In [31]:
import altair as alt
import pandas as pd
import geopandas as gpd
import geopy.distance
import numpy as np
from shapely.geometry import Point, LineString, shape

#### Build a basic gazetteer

In [2]:
locations = [['Ferry Building', 37.7955, -122.3936],
             ['Oyster Point', 37.6663, -122.3816],
             ['Alameda Main Street', 37.791190, -122.294059],
             ['Jack London Square', 37.794465, -122.280137],
             ['China Basin', 37.778151, -122.387554],
             ['Pier 33', 37.806958, -122.404181],
             ['Fishermans Wharf', 37.809639, -122.412310],
             ['Tiburon', 37.872633, -122.455384],
             ['Sausalito', 37.856148, -122.477527],
             ['Angel Island', 37.868731, -122.435261],
             ['Vallejo', 38.099878, -122.263048],
             ['Berkeley Marina', 37.865907, -122.314625],
             ['Harbor Bay Ferry Terminal', 37.736261, -122.257391],
             ['Larkspur', 37.944862, -122.508427],
             ['Emery Cove', 37.839570, -122.311875],
             ['Richardson Bay Marina', 37.875992, -122.504928]
            ]

locations_df = gpd.GeoDataFrame(locations, columns=['Name', 'LAT', 'LON'])
locations_df['geometry'] = [Point(xy) for xy in zip(locations_df.LON, locations_df.LAT)]

locations_df

,Name,LAT,LON,geometry
0,Ferry Building,37.795500,-122.393600,POINT (-122.39360 37.79550)
1,Oyster Point,37.666300,-122.381600,POINT (-122.38160 37.66630)
2,Alameda Main Street,37.791190,-122.294059,POINT (-122.29406 37.79119)
3,Jack London Square,37.794465,-122.280137,POINT (-122.28014 37.79447)
4,China Basin,37.778151,-122.387554,POINT (-122.38755 37.77815)
5,Pier 33,37.806958,-122.404181,POINT (-122.40418 37.80696)
6,Fishermans Wharf,37.809639,-122.412310,POINT (-122.41231 37.80964)
7,Tiburon,37.872633,-122.455384,POINT (-122.45538 37.87263)
8,Sausalito,37.856148,-122.477527,POINT (-122.47753 37.85615)
9,Angel Island,37.868731,-122.435261,POINT (-122.43526 37.86873)


#### Visualize

In [4]:
bay = gpd.read_file('../project_flask/static/data/small_bay_topo.json')

bay_map = alt.Chart(bay).mark_geoshape(color="#333333").encode().project(
    type='mercator'
).properties(
    width=880,
    height=800
)

location_map = alt.Chart(locations_df).mark_geoshape(color="red").encode(tooltip='Name:N')

bay_map + location_map

alt.LayerChart(...)

#### Load vessel data

In [5]:
bay_traffic = gpd.read_file('/Users/gregorytozzi/MIDS/W209/ais/data/AIS_2015_01_SF_Bay.csv')

In [17]:
bay_traffic.LAT = pd.to_numeric(bay_traffic.LAT)
bay_traffic.LON = pd.to_numeric(bay_traffic.LON)

In [18]:
bay_traffic.geometry = [Point(xy) for xy in zip(bay_traffic.LON, bay_traffic.LAT)]

In [21]:
bay_traffic.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,geometry
0,367350050,2015-01-01T00:02:49,37.81690,-122.40924,8.5,-85.9,511.0,ALCATRAZ CLIPPER,,WDE4199,1012,under way using engine,38.8300,10.5000,0.0000,,POINT (-122.40924 37.81690)
1,367350050,2015-01-01T00:03:50,37.81883,-122.41103,8.6,-80.2,511.0,ALCATRAZ CLIPPER,,WDE4199,1012,under way using engine,38.8300,10.5000,0.0000,,POINT (-122.41103 37.81883)
2,367350050,2015-01-01T00:01:09,37.81380,-122.40622,8.2,-90.9,511.0,ALCATRAZ CLIPPER,,WDE4199,1012,under way using engine,38.8300,10.5000,0.0000,,POINT (-122.40622 37.81380)
3,366985270,2015-01-01T00:00:08,37.82351,-122.41542,11.8,121.1,120.0,HARBOR EMPEROR,,WCY6430,1012,under way using engine,25.3000,9.8000,0.0000,,POINT (-122.41542 37.82351)
4,367589090,2015-01-01T00:00:28,37.49269,-122.51851,5.9,141.0,511.0,AVENTURA,,WDG9567,1019,,11.1600,0.0000,0.0000,,POINT (-122.51851 37.49269)
